# Optional Lab - Neurons and Layers (PyTorch Version)
In this lab we will explore the inner workings of neurons/units and layers. In particular, the lab will draw parallels to the models you have mastered in Course 1, the regression/linear model and the logistic model. The lab will introduce PyTorch and demonstrate how these models are implemented in that framework.
<figure>
   <img src="./images/C2_W1_NeuronsAndLayers.png"  style="width:540px;height:200px;" >
</figure>


## Packages
**PyTorch**  
PyTorch is an open-source machine learning framework developed by Facebook's AI Research lab. This course will be using PyTorch instead of TensorFlow/Keras.


In [ ]:
import sys, os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

#region 資料載入
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None

def import_data_from_github():
    import urllib.request, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    def get_data_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/data"
        else:
            return Path.cwd() / "data"

    def get_images_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return f"/content/images"
        else:
            return Path.cwd() / "images"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    os.makedirs(UTILS_DIR, exist_ok=True)

    REPO_DIR = "Machine-Learning-Lab"
    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    for file in ["lab_utils_common_nn.py", "lab_neurons_utils.py", "deeplearning.mplstyle"]:
        urllib.request.urlretrieve(f"{BASE}/utils/{file}", f"{UTILS_DIR}/{file}")

repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    
os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.lab_utils_common_nn import dlc
from utils.lab_neurons_utils import plt_prob_1d, sigmoidnp, plt_linear, plt_logistic
plt.style.use('utils/deeplearning.mplstyle')

# 設定隨機種子以便重現結果
torch.manual_seed(1)
np.random.seed(1)
print("✅ 匯入模組及設定繪圖樣式")
#endregion 資料載入

## Neuron without activation - Regression/Linear Model


### DataSet
We'll use an example from Course 1, linear regression on house prices.


In [ ]:
X_train = np.array([[1.0], [2.0]], dtype=np.float32)
Y_train = np.array([[300.0], [500.0]], dtype=np.float32)

fig, ax = plt.subplots(1,1)
ax.scatter(X_train, Y_train, marker='x', c='r', label="Data Points")
ax.legend( fontsize='xx-large')
ax.set_ylabel('Price (in 1000s of dollars)', fontsize='xx-large')
ax.set_xlabel('Size (1000 sqft)', fontsize='xx-large')
plt.show()


### Regression/Linear Model 
The function implemented by a neuron with no activation is the same as in Course 1, linear regression:
$$ f_{\mathbf{w},b}(x^{(i)}) = \mathbf{w}\cdot x^{(i)} + b \tag{1}$$


We can define a layer with one neuron or unit and compare it to the familiar linear regression function.


In [ ]:
# PyTorch 中使用 nn.Linear 來建立線性層
linear_layer = nn.Linear(in_features=1, out_features=1)


Let's examine the weights.


In [ ]:
# 在 PyTorch 中，權重在建立時就會初始化
print(f"w = {linear_layer.weight.data.numpy()}, b = {linear_layer.bias.data.numpy()}")


The weights are initialized randomly. Let's try the model on one example in `X_train`. Note, the input must be a PyTorch tensor.


In [ ]:
# 將 numpy array 轉換為 PyTorch tensor
X_tensor = torch.from_numpy(X_train[0].reshape(1,1))
with torch.no_grad():
    a1 = linear_layer(X_tensor)
print(a1)


The result is a tensor with a shape of (1,1) or one entry.   
Now let's look at the weights and bias. Let's set them to some known values.


In [ ]:
set_w = np.array([[200]], dtype=np.float32)
set_b = np.array([100], dtype=np.float32)

# 在 PyTorch 中設定權重和偏差
linear_layer.weight.data = torch.from_numpy(set_w)
linear_layer.bias.data = torch.from_numpy(set_b)

print(f"w = {linear_layer.weight.data.numpy()}, b = {linear_layer.bias.data.numpy()}")


Let's compare equation (1) to the layer output.


In [ ]:
with torch.no_grad():
    a1 = linear_layer(X_tensor)
print(a1)
alin = np.dot(set_w, X_train[0].reshape(1,1)) + set_b
print(alin)


They produce the same values!
Now, we can use our linear layer to make predictions on our training data.


In [ ]:
# 使用 PyTorch 模型進行預測
X_train_tensor = torch.from_numpy(X_train)
with torch.no_grad():
    prediction_torch = linear_layer(X_train_tensor).numpy()

# 使用 NumPy 進行預測
prediction_np = np.dot(X_train, set_w.T) + set_b


In [ ]:
plt_linear(X_train, Y_train, prediction_torch, prediction_np)


## Neuron with Sigmoid activation
The function implemented by a neuron/unit with a sigmoid activation is the same as in Course 1, logistic  regression:
$$ f_{\mathbf{w},b}(x^{(i)}) = g(\mathbf{w}x^{(i)} + b) \tag{2}$$
where $$g(x) = sigmoid(x)$$ 

Let's set $w$ and $b$ to some known values and check the model.


### DataSet
We'll use an example from Course 1, logistic regression.


In [ ]:
X_train = np.array([0., 1, 2, 3, 4, 5], dtype=np.float32).reshape(-1,1)
Y_train = np.array([0,  0, 0, 1, 1, 1], dtype=np.float32).reshape(-1,1)


In [ ]:
pos = Y_train == 1
neg = Y_train == 0

fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.scatter(X_train[pos], Y_train[pos], marker='x', s=80, c = 'red', label="y=1")
ax.scatter(X_train[neg], Y_train[neg], marker='o', s=100, label="y=0", facecolors='none', 
              edgecolors=dlc["dlblue"],lw=3)

ax.set_ylim(-0.08,1.1)
ax.set_ylabel('y', fontsize=12)
ax.set_xlabel('x', fontsize=12)
ax.set_title('one variable plot')
ax.legend(fontsize=12)
plt.show()


### Logistic Neuron
We can implement a 'logistic neuron' by adding a sigmoid activation. The function of the neuron is then described by (2) above.   
This section will create a PyTorch Model that contains our logistic layer to demonstrate an alternate method of creating models. PyTorch is most often used to create multi-layer models. The [Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) model is a convenient means of constructing these models.


In [ ]:
# 使用 Sequential 建立模型
model = nn.Sequential(
    nn.Linear(1, 1),
    nn.Sigmoid()
)


Let's look at the model structure. There is only one layer in this model and that layer has only one unit. The unit has two parameters, $w$ and $b$.


In [ ]:
print(model)
print("\nModel parameters:")
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")


In [ ]:
# 獲取第一層（Linear layer）
logistic_layer = model[0]
w = logistic_layer.weight.data.numpy()
b = logistic_layer.bias.data.numpy()
print(w, b)
print(w.shape, b.shape)


Let's set the weight and bias to some known values.


In [ ]:
set_w = np.array([[2]], dtype=np.float32)
set_b = np.array([-4.5], dtype=np.float32)

# 設定權重
logistic_layer.weight.data = torch.from_numpy(set_w)
logistic_layer.bias.data = torch.from_numpy(set_b)

print(f"w = {logistic_layer.weight.data.numpy()}, b = {logistic_layer.bias.data.numpy()}")


Let's compare equation (2) to the layer output.


In [ ]:
X_test = torch.from_numpy(X_train[0].reshape(1,1))
with torch.no_grad():
    a1 = model(X_test).numpy()
print(a1)

alog = sigmoidnp(np.dot(set_w, X_train[0].reshape(1,1)) + set_b)
print(alog)


They produce the same values!
Now, we can use our logistic layer and NumPy model to make predictions on our training data.


In [ ]:
# 創建一個包裝函數以便與繪圖工具相容
class ModelWrapper:
    def __init__(self, model):
        self.model = model
    
    def predict(self, X):
        X_tensor = torch.from_numpy(X.astype(np.float32))
        with torch.no_grad():
            return self.model(X_tensor).numpy()

wrapped_model = ModelWrapper(model)
plt_logistic(X_train, Y_train, wrapped_model, set_w, set_b, pos, neg)


The shading above reflects the output of the sigmoid which varies from 0 to 1.


# Congratulations!
You built a very simple neural network using PyTorch and have explored the similarities of a neuron to the linear and logistic regression from Course 1.
